# **Sliding Puzzle Problem**
## **Introduction**

The objective of the lab is to efficiently solve a general \(n^2 - 1\) puzzle problem, such as the Gem Puzzle or Mystic Square, using path-search algorithms. This means finding a sequence of actions to transform a random initial state into a goal state, optimizing:

- **Quality**: The number of actions in the solution.
- **Cost**: The total number of nodes evaluated during the search.
- **Efficiency**: A trade-off between quality and cost.

---

### Import Libraries and Constants

The necessary libraries are imported, and constants are defined for puzzle dimensions and randomization steps.


In [12]:
from collections import namedtuple
from random import choice
from heapq import heappush, heappop
from tqdm.auto import tqdm
import numpy as np

# Puzzle Configuration
RANDOMIZE_STEPS = 100_000  # Number of random moves to shuffle the puzzle
PUZZLE_DIM = 4  # Dimension of the puzzle (e.g., 4 for the 15-puzzle)
# Define an Action as a swap between two positions
Action = namedtuple('Action', ['pos1', 'pos2'])

### Puzzle Utility Functions
Generate Available Actions

Defines moves for the empty tile.


In [13]:
def available_actions(state: np.ndarray) -> list[Action]:
    """
    Find all valid moves for the empty tile (0).

    Args:
        state (np.ndarray): Current puzzle state.

    Returns:
        list[Action]: List of possible actions as swaps.
    """
    x, y = np.argwhere(state == 0)[0]
    actions = []
    if x > 0:
        actions.append(Action((x, y), (x - 1, y)))  # Move Up
    if x < PUZZLE_DIM - 1:
        actions.append(Action((x, y), (x + 1, y)))  # Move Down
    if y > 0:
        actions.append(Action((x, y), (x, y - 1)))  # Move Left
    if y < PUZZLE_DIM - 1:
        actions.append(Action((x, y), (x, y + 1)))  # Move Right
    return actions

Execute an Action

Performs the swap for a selected action.

In [14]:
def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    """
    Execute a move and return the new puzzle state.

    Args:
        state (np.ndarray): Current puzzle state.
        action (Action): Action to perform.

    Returns:
        np.ndarray: New puzzle state after the move.
    """
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state



In [15]:
def manhattan_distance(state: np.ndarray) -> int:
    """
    Compute the Manhattan distance for the puzzle state.

    Args:
        state (np.ndarray): Current puzzle state.

    Returns:
        int: Total Manhattan distance of the state.
    """
    distance = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:  # Ignore the empty tile
                target_x, target_y = divmod(value - 1, PUZZLE_DIM)
                distance += abs(x - target_x) + abs(y - target_y)
    return distance

In [16]:
def is_solvable(state: np.ndarray) -> bool:
    """Check if the puzzle state is solvable."""
    flat_state = state.flatten()
    inversions = 0
    for i in range(len(flat_state)):
        for j in range(i + 1, len(flat_state)):
            if flat_state[i] and flat_state[j] and flat_state[i] > flat_state[j]:
                inversions += 1
    return inversions % 2 == 0

In [17]:
def print_state(state: np.ndarray):
    """Print the puzzle state in a readable format."""
    for row in state:
        print(' '.join(f"{num:2}" for num in row))
    print()

### Randomize Puzzle State
Generates a randomized puzzle state by performing a number of random moves.

In [18]:
def randomize_puzzle() -> np.ndarray:
    """Generate a randomized puzzle state by performing random moves."""
    state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
    for _ in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
        state = do_action(state, choice(available_actions(state)))
    return state

## Bidirectional A* Search Algorithm

Implement the main bidirectional A* search algorithm by coordinating forward and backward searches.


In [19]:
def a_star_search(initial_state: np.ndarray, goal_state: np.ndarray, forward=True):
    """
    Performs an A* search in the specified direction (forward or backward).

    Args:
        initial_state (np.ndarray): The initial state for the search.
        goal_state (np.ndarray): The goal state for the search.
        forward (bool): Search direction. True for forward, False for backward.

    Returns:
        tuple: (open_set, came_from, g_score)
    """
    open_set = []
    heappush(open_set, (manhattan_distance(initial_state), initial_state.tobytes()))
    came_from = {initial_state.tobytes(): None}
    g_score = {initial_state.tobytes(): 0}

    return open_set, came_from, g_score

In [20]:
def bidirectional_a_star(initial_state: np.ndarray):
    """Performs a bidirectional A* search to solve the puzzle."""
    goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

    # Initialize forward and backward searches
    open_forward, came_from_forward, g_score_forward = a_star_search(initial_state, goal_state, forward=True)
    open_backward, came_from_backward, g_score_backward = a_star_search(goal_state, initial_state, forward=False)

    explored_forward = set()
    explored_backward = set()

    nodes_evaluated = 0
    meeting_node = None

    while open_forward and open_backward:
        # Expand forward search
        if open_forward:
            f_fwd, current_fwd = heappop(open_forward)
            current_state_fwd = np.frombuffer(current_fwd, dtype=int).reshape((PUZZLE_DIM, PUZZLE_DIM))
            explored_forward.add(current_fwd)

            if current_fwd in explored_backward:
                meeting_node = current_fwd
                break

            for action in available_actions(current_state_fwd):
                neighbor = do_action(current_state_fwd, action)
                neighbor_bytes = neighbor.tobytes()
                tentative_g = g_score_forward[current_fwd] + 1

                if neighbor_bytes not in g_score_forward or tentative_g < g_score_forward[neighbor_bytes]:
                    g_score_forward[neighbor_bytes] = tentative_g
                    f_score = tentative_g + manhattan_distance(neighbor)
                    heappush(open_forward, (f_score, neighbor_bytes))
                    came_from_forward[neighbor_bytes] = current_fwd
                    nodes_evaluated += 1

        # Expand backward search
        if open_backward:
            f_bwd, current_bwd = heappop(open_backward)
            current_state_bwd = np.frombuffer(current_bwd, dtype=int).reshape((PUZZLE_DIM, PUZZLE_DIM))
            explored_backward.add(current_bwd)

            if current_bwd in explored_forward:
                meeting_node = current_bwd
                break

            for action in available_actions(current_state_bwd):
                neighbor = do_action(current_state_bwd, action)
                neighbor_bytes = neighbor.tobytes()
                tentative_g = g_score_backward[current_bwd] + 1

                if neighbor_bytes not in g_score_backward or tentative_g < g_score_backward[neighbor_bytes]:
                    g_score_backward[neighbor_bytes] = tentative_g
                    f_score = tentative_g + manhattan_distance(neighbor)
                    heappush(open_backward, (f_score, neighbor_bytes))
                    came_from_backward[neighbor_bytes] = current_bwd
                    nodes_evaluated += 1

    if meeting_node is None:
        return None, nodes_evaluated

    # Reconstruct the path
    path_forward = []
    node = meeting_node
    while node is not None:
        path_forward.append(node)
        node = came_from_forward.get(node)

    path_backward = []
    node = came_from_backward.get(meeting_node)
    while node is not None:
        path_backward.append(node)
        node = came_from_backward.get(node)

    full_path = path_forward[::-1] + path_backward

    return full_path, nodes_evaluated

In [21]:
def reconstruct_full_path(path_bytes: list) -> list:
    """Reconstructs the complete solution path from state bytes."""
    return [np.frombuffer(state_bytes, dtype=int).reshape((PUZZLE_DIM, PUZZLE_DIM)) for state_bytes in path_bytes]


In [22]:
def main():
    """Main function to execute the puzzle solver for multiple puzzle dimensions."""
    # List of puzzle sizes to test
    puzzle_sizes = [3, 4, 5]  # Example: 3x3, 4x4, 5x5 puzzles

    for size in puzzle_sizes:
        global PUZZLE_DIM  # Adjust the global puzzle dimension
        PUZZLE_DIM = size

        print(f"\n{'=' * 40}")
        print(f"Solving {PUZZLE_DIM}x{PUZZLE_DIM} Puzzle")
        print(f"{'=' * 40}\n")

        # Randomize the puzzle until a solvable state is obtained
        while True:
            state = randomize_puzzle()
            if is_solvable(state):
                break
            print("Generated unsolvable state, trying again...")

        print("Initial state:")
        print_state(state)
        print(f"Solvable: {is_solvable(state)}")

        # Perform bidirectional A* search
        solution, cost = bidirectional_a_star(state)

        if solution:
            quality = len(solution) - 1  # Number of moves
            efficiency = quality / cost if cost > 0 else 0

            print(f"\nSolution found in {quality} steps!")
            print(f"Quality (Number of steps): {quality}")
            print(f"Cost (Nodes evaluated): {cost}")
            print(f"Efficiency (Quality / Cost): {efficiency:.6f}")

            print("\nGoal state (solution):")
            final_state = reconstruct_full_path(solution)[-1]
            print_state(final_state)
        else:
            print("No solution found!")


In [ ]:
if __name__ == "__main__":
    main()


Solving 3x3 Puzzle



Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

Initial state:
 0  3  1
 6  7  5
 4  8  2

Solvable: True

Solution found in 22 steps!
Quality (Number of steps): 22
Cost (Nodes evaluated): 296
Efficiency (Quality / Cost): 0.074324

Goal state (solution):
 1  2  3
 4  5  6
 7  8  0


Solving 4x4 Puzzle



Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

Initial state:
 2  5  1 14
 9  0  7 11
10  8 15  4
13  3 12  6

Solvable: True

Solution found in 52 steps!
Quality (Number of steps): 52
Cost (Nodes evaluated): 276746
Efficiency (Quality / Cost): 0.000188

Goal state (solution):
 1  2  3  4
 5  6  7  8
 9 10 11 12
13 14 15  0


Solving 5x5 Puzzle



Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

Initial state:
10 17  9 23 21
 4 19  5  2  3
11 12  7  1 14
24 20 15  0 18
22 13  6  8 16

Solvable: True
